In [1]:
import json
import pandas as pd
from pandas import json_normalize
from datasets import load_from_disk
import string
import re
import warnings
warnings.filterwarnings('ignore')

# Reader 데이터 경로

In [2]:
dataset = load_from_disk("input/data/train_dataset")
test_dataset=load_from_disk("input/data/test_dataset")

train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset=test_dataset['validation']

train_df=pd.DataFrame(train_dataset)
valid_df=pd.DataFrame(valid_dataset)
test_df=pd.DataFrame(test_dataset)

# Retrieval 데이터 경로

In [3]:
file_path = "input/data/wikipedia_documents.json"

with open(file_path, "r") as json_file:
    json_data = json.load(json_file)

# SOTA , 내 예측 경로

In [4]:
sota_path="trash/ny.json"
my_pre_path="input/level2-mrc-level2-nlp-09/outputs/ae1e-6wu0bs16power2lr9e-6e5_re_check2000_topk40/predictions.json"

with open(sota_path, "r") as json_file:
    sota= json.load(json_file)
with open(my_pre_path, "r") as json_file:
    my_pre= json.load(json_file)

# test_df 비교

### 후처리 함수 : [Korquad 공식 사이트의 evaluation script(채점 코드) 참고](https://korquad.github.io/KorQuad%201.0/)
#### 후처리 함수 적용과정 : 정의한 불필요한 기호 제거 -> lower -> string.punctuation 제거 -> 공백 제거

In [5]:
import string
print(set(string.punctuation))

{']', '#', '@', '{', '*', ':', '?', '&', "'", '>', '^', '}', '"', '-', '/', '$', '<', '!', '.', '[', '+', '~', ',', '%', '\\', '=', '_', '(', ';', '|', '`', ')'}


In [6]:
import re
def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

#### 예시

In [7]:
normalize_answer('간석기(마제석기)')

'간석기 마제석기'

## SOTA 제출과 비교 및 후처리한 항목 추가

In [8]:
sota_list=list(sota.values())
my_pre_list=list(my_pre.values())

diff=[]
for js in sota_list:
    if js not in my_pre_list:
        diff.append(sota_list.index(js))
        
sota_diff_list=[]
my_pre_diff_list=[]

for df in diff:
    sota_diff_list.append(sota_list[df])
    my_pre_diff_list.append(my_pre_list[df])

sota_diff=test_df.iloc[diff]
sota_diff["SOTA_ans"]=sota_diff_list
sota_diff["my_ans"]=my_pre_diff_list
sota_diff["SOTA_ans_len"]=list(map(len,sota_diff_list))
sota_diff["my_ans_len"]=list(map(len,my_pre_diff_list))
sota_diff['post_process(SOTA)']=sota_diff['SOTA_ans'].apply(lambda x: normalize_answer(x))
sota_diff['post_process(my)']=sota_diff['my_ans'].apply(lambda x: normalize_answer(x))

sota_diff=sota_diff.drop(['id'],axis=1)

In [9]:
print("#############################################################")
print(f"SOTA와 비교하여 다른 답변의 개수는 {len(sota_diff)}개 입니다.")
print("#############################################################")
sota_diff.sort_values(by=["SOTA_ans_len","my_ans_len"],ascending=False).head(20)

#############################################################
SOTA와 비교하여 다른 답변의 개수는 229개 입니다.
#############################################################


,question,SOTA_ans,my_ans,SOTA_ans_len,my_ans_len,post_process(SOTA),post_process(my)
337,외국인 근로자 고용 등에 대한 법률'의 목적은?,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는 것,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는,48,46,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는 것,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는
543,인민군이 세르비아를 확장하는 쪽으로 돌아서게 만든 사람은?,슬로보단 밀로셰비치의 통제 하로 들어가게 된다. 밀로셰비치,밀로셰비치,32,5,슬로보단 밀로셰비치의 통제 하로 들어가게 된다 밀로셰비치,밀로셰비치
80,연합 함대가 국지 경비 부대와 해상 호위 부대의 인재 육성을 경시해 일어난 결과는?,"해상 교통, 보급 확보에 충당해야 할 함정과 인력의 부족","해군의 중요한 임무였던 해상 교통, 보급 확보에 충당해야 할 함정과 인력의 부족",31,44,해상 교통 보급 확보에 충당해야 할 함정과 인력의 부족,해군의 중요한 임무였던 해상 교통 보급 확보에 충당해야 할 함정과 인력의 부족
342,1대부터 4대 칼리파를 통틀어 부른 명칭은?,정통 칼리파(Al-Khalifah Ar-Rashid),정통 칼리파,29,6,정통 칼리파 alkhalifah arrashid,정통 칼리파
89,단색 배경에 추상적인 디자인을 쓰는 양식이 유행하던 시기는?,아우구스투스 시대(기원전 27년 ~ 기원후 14년),아우구스투스 시대,28,9,아우구스투스 시대 기원전 27년 기원후 14년,아우구스투스 시대
569,하나를 철거하면 열을 건다' 운동에 사용된 플랜카드의 내용은?,'나는 참된 보통 선거를 원한다 (我要真普選)','나는 참된 보통 선거를 원한다',26,18,나는 참된 보통 선거를 원한다 我要真普選,나는 참된 보통 선거를 원한다
230,아일랜드의 자원 공급 불안과 지구 온난화에 대한 해결방안은?,풍력이나 조력 발전 등 대체 에너지의 개발,대체 에너지의 개발,23,10,풍력이나 조력 발전 등 대체 에너지의 개발,대체 에너지의 개발
378,보디 스타킹에서 봉제되지 않은 부분은?,오른쪽 허벅지 가로 15cm 세로 20cm,연성 수지,23,5,오른쪽 허벅지 가로 15cm 세로 20cm,연성 수지
491,삼탄당 인산 이성질화효소 결핍증은 주로 어떤 물질의 돌연변이로 일어나나요?,104번 위치의 글루탐산이 아스파르트산,아스파르트산,21,6,104번 위치의 글루탐산이 아스파르트산,아스파르트산
583,인간에게 불타는 느낌의 아픔을 주는 것은?,솔레놉신이 있다. 인간에게는 솔레놉신이,백루검,21,3,솔레놉신이 있다 인간에게는 솔레놉신이,백루검


# 전체

In [10]:
test_df["SOTA_ans"]=sota_list
test_df["my_ans"]=my_pre_list
test_df["SOTA_ans_len"]=test_df["SOTA_ans"].apply(lambda x: len(x))
test_df["my_ans_len"]=test_df["my_ans"].apply(lambda x: len(x))
test_df['post_process(SOTA)']=test_df['SOTA_ans'].apply(lambda x: normalize_answer(x))
test_df['post_process(my)']=test_df['my_ans'].apply(lambda x: normalize_answer(x))
test_df.sort_values(by=["SOTA_ans_len","my_ans_len"],ascending=False)

,question,id,SOTA_ans,my_ans,SOTA_ans_len,my_ans_len,post_process(SOTA),post_process(my)
337,외국인 근로자 고용 등에 대한 법률'의 목적은?,mrc-0-004594,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는 것,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는,48,46,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는 것,교육 및 문화 교류를 통해 미국 사람들과 다른 국가 사람들 간의 상호 이해를 높이는
543,인민군이 세르비아를 확장하는 쪽으로 돌아서게 만든 사람은?,mrc-1-000524,슬로보단 밀로셰비치의 통제 하로 들어가게 된다. 밀로셰비치,밀로셰비치,32,5,슬로보단 밀로셰비치의 통제 하로 들어가게 된다 밀로셰비치,밀로셰비치
80,연합 함대가 국지 경비 부대와 해상 호위 부대의 인재 육성을 경시해 일어난 결과는?,mrc-0-003540,"해상 교통, 보급 확보에 충당해야 할 함정과 인력의 부족","해군의 중요한 임무였던 해상 교통, 보급 확보에 충당해야 할 함정과 인력의 부족",31,44,해상 교통 보급 확보에 충당해야 할 함정과 인력의 부족,해군의 중요한 임무였던 해상 교통 보급 확보에 충당해야 할 함정과 인력의 부족
342,1대부터 4대 칼리파를 통틀어 부른 명칭은?,mrc-0-004862,정통 칼리파(Al-Khalifah Ar-Rashid),정통 칼리파,29,6,정통 칼리파 alkhalifah arrashid,정통 칼리파
89,단색 배경에 추상적인 디자인을 쓰는 양식이 유행하던 시기는?,mrc-0-003364,아우구스투스 시대(기원전 27년 ~ 기원후 14년),아우구스투스 시대,28,9,아우구스투스 시대 기원전 27년 기원후 14년,아우구스투스 시대
...,...,...,...,...,...,...,...,...
114,잉글랜드에서 보리와 함께 가장 많이 경작하는 작물은 무엇인가요?,mrc-0-004538,밀,밀,1,1,밀,밀
322,신돈의 어미니는 어느 곳에 소속된 노비인가?,mrc-1-001035,절,절,1,1,절,절
346,크라테로스를 사망에 이르게 한 동물은?,mrc-1-000691,말,말,1,1,말,말
372,수술대 하부에 꿀샘이 존재하면 꽃밥은 어디를 향해 있는가?,mrc-0-000780,밖,밖,1,1,밖,밖


## 얻은 인사이트

### 1. Korquad 기준으로는 위의 sub->소문자->remove punc->공백제거
- " << " 기호처럼 부등식을 두개합친 것은 제거가 가능하지만, 밑의 악령 예시처럼 정의되지 않은 기호는 처리 되지 않음
- 채점 기준을 명확히 알 수 없기에 함부로 다룰 수 없음

![aa](https://user-images.githubusercontent.com/46811558/167540135-143de64b-e694-4110-beba-7d0dbf230c90.png)

### 2. 밑의 예시처럼 명확하지 않지만 직관적으로 틀린 답이 있을 경우 처리
- 첫번째, 직접 json을 보며 제거 -> 치팅이라 X
- 두번째, 형태소 분석기 

In [12]:
josa=[543,583]
test_df.iloc[josa]

,question,id,SOTA_ans,my_ans,SOTA_ans_len,my_ans_len,post_process(SOTA),post_process(my)
543,인민군이 세르비아를 확장하는 쪽으로 돌아서게 만든 사람은?,mrc-1-000524,슬로보단 밀로셰비치의 통제 하로 들어가게 된다. 밀로셰비치,밀로셰비치,32,5,슬로보단 밀로셰비치의 통제 하로 들어가게 된다 밀로셰비치,밀로셰비치
583,인간에게 불타는 느낌의 아픔을 주는 것은?,mrc-0-001659,솔레놉신이 있다. 인간에게는 솔레놉신이,백루검,21,3,솔레놉신이 있다 인간에게는 솔레놉신이,백루검


### 3. 추가적으로 가능한 시도
- 앙상블1 : sota의 json을 모아서 후처리 함수 통과한 이후에의 hard voting?
    - 추후작성 : [앙상블 실험관련](https://github.com/boostcampaitech3/level2-mrc-level2-nlp-09/issues/45)에서 성능향상 확인 가능
    - 답을 정확히 알 수 없기에 명확한 근거는 없지만, 동일 세팅하에 f1 점수 0.11점 상승
- 앙상블2 : nbest_prediction 파일을 참고하여 후처리 한 후 소프트 보팅 앙상블
    - Ex: 5개의 답변 -> 지구 : 0.3,0.3,0.2 / 화성 0.4,0.7 -> 화성으로 출력
- 룰베이스  
    ![13](https://user-images.githubusercontent.com/46811558/167543485-7b30bcde-b25a-4044-b6f9-5987c7e5567a.png)
- 형태소 분석기
    - [소개글](https://couplewith.tistory.com/entry/Python-KoNLPy-%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-%EB%B9%84%EA%B5%90-Komoran-Okt-Kkma) -> Komoran ,Okt , Khma

In [13]:
from konlpy.tag import Komoran, Kkma, Okt, Hannanum
komoran = Komoran()
kkma = Kkma()
okt = Okt()
hannanum=Hannanum()

In [14]:
sentence='슬로보단 밀로셰비치의 통제 하로 들어가게 된다. 밀로셰비치'
class List(list): 
    def __str__(self): 
        return "[" + ", ".join(["%s" % x for x in self]) + "]"
print(List(komoran.morphs(sentence)))
print(List(hannanum.morphs(sentence)))
print(List(okt.morphs(sentence)))
print(List(kkma.morphs(sentence)))
print("------------")
print(komoran.morphs(sentence))
print(hannanum.morphs(sentence))
print(okt.morphs(sentence))
print(kkma.morphs(sentence))

[슬로보단 밀로셰비치, 의, 통제, 하, 로, 들어가, 게, 되, ㄴ다, ., 밀로셰비치]
[슬로보단, 밀로셰비치, 의, 통제, 하, 로, 들, 어, 가, 게, 되, ㄴ다, ., 밀로셰비치]
[슬로, 보단, 밀로셰비치, 의, 통제, 하, 로, 들어가게, 된다, ., 밀로셰비치]
[슬, ㄹ, 로, 보다, ㄴ, 밀, 로, 셰, 비치, 의, 통제, 하, 로, 들어가, 게, 되, ㄴ다, ., 밀, 로, 셰, 비치]
------------
['슬로보단 밀로셰비치', '의', '통제', '하', '로', '들어가', '게', '되', 'ㄴ다', '.', '밀로셰비치']
['슬로보단', '밀로셰비치', '의', '통제', '하', '로', '들', '어', '가', '게', '되', 'ㄴ다', '.', '밀로셰비치']
['슬로', '보단', '밀로셰비치', '의', '통제', '하', '로', '들어가게', '된다', '.', '밀로셰비치']
['슬', 'ㄹ', '로', '보다', 'ㄴ', '밀', '로', '셰', '비치', '의', '통제', '하', '로', '들어가', '게', '되', 'ㄴ다', '.', '밀', '로', '셰', '비치']


### ETC1 : Pororo(NER)
- 질문에서 "누구?,어디?,것?,이유?,사람은?" 등을 추출하여 난이도 별로 질문 나누기
    - 추후에 curriculum learning 가능?
    - 국가 -> 프랑스군 vs 프랑스 처럼 아닌 답변 걸러내기

In [15]:
from pororo import Pororo

Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s

In [16]:
ner = Pororo(task="ner", lang="kor")
ner('슬로보단 밀로셰비치의 통제 하로 들어가게 된다. 밀로셰비치')

[Korean Sentence Splitter]: Initializing Pynori...


[('슬로보', 'PERSON'),
 ('단', 'O'),
 (' ', 'O'),
 ('밀로셰비치', 'PERSON'),
 ('의', 'O'),
 (' ', 'O'),
 ('통제', 'O'),
 (' ', 'O'),
 ('하로', 'O'),
 (' ', 'O'),
 ('들어가게', 'O'),
 (' ', 'O'),
 ('된다.', 'O'),
 (' ', 'O'),
 ('밀로셰비치', 'PERSON')]

In [17]:
ner('인민군이 세르비아를 확장하는 쪽으로 돌아서게 만든 사람은?')

[('인민군', 'ORGANIZATION'),
 ('이', 'O'),
 (' ', 'O'),
 ('세르비아', 'COUNTRY'),
 ('를', 'O'),
 (' ', 'O'),
 ('확장하는', 'O'),
 (' ', 'O'),
 ('쪽으로', 'O'),
 (' ', 'O'),
 ('돌아서게', 'O'),
 (' ', 'O'),
 ('만든', 'O'),
 (' ', 'O'),
 ('사람은?', 'O')]

### ETC2 : Pororo(Summarization)
- 위키 Retrieval 데이터 전처리 -> summarization 하여 답변 탐색
    - 메모리 오류& 문서에서의 답변 사라지는 문제 때문에 제출 X

In [18]:
summ = Pororo(task="summarization", model="abstractive", lang="ko")

In [19]:
summ('이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.두 목록은 모두 가나다 순이다.일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.')

'명 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있으며 이 목록은 명료화를 위해 두 부분으로 나뉘어 있으며, 두번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.'

## [wiki 전처리](https://github.com/boostcampaitech3/level2-mrc-level2-nlp-09/blob/deff4a0e1d4e0977333f97fb7210080e75720843/elastic_setting.py#L39)

In [20]:
def preprocess(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":%&《》〈〉''㈜·\-\'+\s一-龥サマーン]", "", text)  # サマーン 는 predictions.json에 있었음
    text = re.sub(r"\s+", " ", text).strip()  # 두 개 이상의 연속된 공백을 하나로 치환
    # text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":%&《》〈〉''㈜·\-\'+\s一-龥]", "", text)

    return text

In [21]:
import json
import pandas as pd
from pandas import json_normalize

file_path = "input/data/wikipedia_documents.json"

with open(file_path, "r",encoding='UTF-8') as json_file:
    json_data = json.load(json_file)

summ_json_path = "pororo.json"

# with open(del_json_path, 'w', encoding='utf-8') as file:
#     json.dump(json_da, file,ensure_ascii=False)

In [27]:
json_list=[]
count=0

In [28]:
for i in range(len(json_data)):
    json_list.append(summ(preprocess(json_data[str(i)]['text'])))
    count+=1
    if count>1000:
        break
    # json_data[str(i)]['text']=summ(preprocess(json_data[str(i)]['text']))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [68]:
json_data['0']['text']

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [69]:
summ(preprocess(json_data['0']['text']))

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있으며 이 목록은 명료화를 위해 두 부분으로 나뉘어 있으며, 이 목록은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.'